In [1]:
from bprH import bprH, adv_index, save_model, load_model
import pandas as pd
from sklearn.model_selection import KFold
import numpy as np

In [15]:
data = [['U1','I1','V'], ['U1','I1','P'], ['U1','I3','V'], ['U1','I4','V'],
        ['U2','I2','V'], ['U2','I2','P'], ['U2','I1','V'],
        ['U3','I1','V'], ['U3','I1','P'], ['U3','I2','V'], ['U3','I4','V'], ['U3','I4','P'],
        ['U4','I2','V'], ['U4','I3','V'], ['U4','I3','P'], ['U4','I4','V'],
        ['U5','I1','V'], ['U5','I3','V'], ['U5','I2','V'], ['U5','I2','P']
        ]
data = pd.DataFrame(data, columns=['UserID', 'ItemID', 'Action'])

test = [['U1','I3','P'], ['U2','I1','P'], ['U3','I2','P'], ['U4','I4','P'], ['U5','I1','P']]
test = pd.DataFrame(test, columns=['UserID', 'ItemID', 'Action'])

In [16]:
# start a BPRH model
model = bprH(dim=3, omega=1, rho=1, lambda_u=0.1, lambda_v=0.1, lambda_b=0.1, gamma=0.008, num_iter=10000, random_state=20200704)

In [17]:
model.fit(X=data, eval_X=test, original_item_list=['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12'], original_user_list=['U1', 'U2', 'U3', 'U4', 'U5'], saved_path='data/small-item-set-coselection.pkl')

Calculate auxiliary-target correlation
Generate Itemset Coselection


ITER 4893:  49%|████▉     | 4894/10000 [02:01<02:05, 40.70it/s, loss=-0.931, precision_10=0.1, precision_5=0, recall_10=1, recall_5=0]        


KeyboardInterrupt: 

In [ ]:
model.predict_estimation(user_to_predict=['U1'], item_to_predict=['I1', 'I2'])

In [6]:
# read sobazaar data
data = pd.read_csv('data/sb_unique_actions_20.csv')
# change column name
data.columns = ['ItemID', 'UserID', 'Action', 'Action_Date', 'Action_Time',
       'SessionID']

data = data.sample(frac=0.1) # TODO: test small data

In [7]:
# the best way is to split the data with a time manner - from sklearn.model_selection import TimeSeriesSplit
# but first let's do it in a simple way

original_item_list = sorted(set(data.ItemID))
original_user_list = sorted(set(data.UserID))

data_without_target = data[data.Action != 'P']
data_only_with_target = data[data.Action == 'P']

In [8]:
# split the data_only_with_target into 5 folds
kf = KFold(n_splits=2,shuffle=True, random_state=0)
kf.get_n_splits(X=data_only_with_target)

2

In [ ]:
cnt = 0
for train_index, test_index in kf.split(X=data_only_with_target):
    # build train and test data
    train_data = pd.concat([data_only_with_target.iloc[train_index], data_without_target])
    test_data = data_only_with_target.iloc[test_index]

    # start a BPRH model
    model = bprH(dim=50, omega=1000, rho=1, lambda_u=0.5, lambda_v=0.5, lambda_b=0.5, gamma=0.01, random_state=20200704, num_iter=10000)

    model.fit(X=train_data, eval_X=test_data, original_item_list=original_item_list, original_user_list=original_user_list, saved_path='data/item-set-coselection-'+str(cnt)+'.pkl')
    cnt += 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value

  3%|▎         | 65/2254 [01:21<52:20,  1.43s/it]

Calculate auxiliary-target correlation
Generate Itemset Coselection


In [7]:
train_data = pd.concat([data_only_with_target.iloc[train_index], data_without_target])
test_data = data_only_with_target.iloc[test_index]

In [ ]:
# start a BPRH model
model = bprH(dim=50, omega=1000, rho=1, lambda_u=0.1, lambda_v=0.1, lambda_b=0.1, gamma=0.001, random_state=20200704)

model.fit(X=train_data, original_item_list=original_item_list, original_user_list=original_user_list, num_iter=10000, saved_path='data/item-set-coselection-'+str(cnt)+'.pkl')
cnt += 1

In [13]:
user_to_predict = sorted(set(test_data.UserID))

In [9]:
model = load_model(load_path='bprH_model.pkl')

In [14]:
model.scoring(ground_truth=test_data, K=5)

Build User's Purchased Item Dict & Rec List



  0%|          | 0/1271 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices